<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/options/calibrated_binomial_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

def stockTree(S, sigma, T, n):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    return [[S * u ** (t - 2 * i) for i in range(t + 1)] for t in range(n + 1)]


def params(r, q, sigma, T, n):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp((r - q) * dt) - d) / (u - d)
    disc = np.exp(-r * dt)
    return u, d, p, disc


def europeanTree(S, K, r, q, sigma, T, n, kind):
    def f(S):
        if kind == "call":
            return np.maximum(np.array(S) - K, 0)
        else:
            return np.maximum(K - np.array(S), 0)

    u, d, p, disc = params(r, q, sigma, T, n)
    ST = [S * u ** (n - 2 * i) for i in range(n + 1)]
    x = f(ST)
    lst = [x]
    while len(x) > 1:
        x = disc * (p * x[:-1] + (1 - p) * x[1:])
        lst.insert(0, x)
    return [list(x) for x in lst], p


def americanTree(S, K, r, q, sigma, T, n, kind):
    def f(S):
        if kind == "call":
            return np.maximum(np.array(S) - K, 0)
        else:
            return np.maximum(K - np.array(S), 0)

    u, d, p, disc = params(r, q, sigma, T, n)
    ST = [S * u ** (n - 2 * i) for i in range(n + 1)]
    x = f(ST)
    lst = [x]
    while len(x) > 1:
        x0 = disc * (p * x[:-1] + (1 - p) * x[1:])
        t = len(x0) - 1
        St = [S * u ** (t - 2 * i) for i in range(t + 1)]
        x = np.maximum(x0, f(St))
        lst.insert(0, x)
    return [list(x) for x in lst], p

In [ ]:
# example parameters

S = 50
K = 50
T = 1
sigma = 0.4
r = 0.02
q = 0.03
n = 5
kind = 'put'

# stock tree

df = pd.DataFrame(stockTree(S, sigma, T, n)).transpose()
df.columns = ['period ' + str(i) for i in range(df.shape[0])]
df

,period 0,period 1,period 2,period 3,period 4,period 5
0,50.0,59.794187,71.506895,85.513933,102.264721,122.296717
1,NaN,41.810085,50.000000,59.794187,71.506895,85.513933
2,NaN,NaN,34.961663,41.810085,50.000000,59.794187
3,NaN,NaN,NaN,29.235002,34.961663,41.810085
4,NaN,NaN,NaN,NaN,24.446358,29.235002
5,NaN,NaN,NaN,NaN,NaN,20.442086


In [ ]:
# European put tree

df = pd.DataFrame(europeanTree(S, K, r, q, sigma, T, n, kind)[0]).transpose()
df.columns = ['period ' + str(i) for i in range(df.shape[0])]
df

,period 0,period 1,period 2,period 3,period 4,period 5
0,8.359248,4.287081,1.347500,0.000000,0.000000,0.000000
1,NaN,11.749801,6.721895,2.459116,0.000000,0.000000
2,NaN,NaN,15.946527,10.256377,4.487756,0.000000
3,NaN,NaN,NaN,20.715317,15.047878,8.189915
4,NaN,NaN,NaN,NaN,25.500280,20.764998
5,NaN,NaN,NaN,NaN,NaN,29.557914


In [ ]:
print('European Put value at date 0:', df.iloc[0,0].round(3))

European Put value at date 0: 8.359


In [ ]:
# American put tree

df = pd.DataFrame(americanTree(S, K, r, q, sigma, T, n, kind)[0]).transpose()
df.columns = ['period ' + str(i) for i in range(df.shape[0])]
df

,period 0,period 1,period 2,period 3,period 4,period 5
0,8.367422,4.287081,1.347500,0.000000,0.000000,0.000000
1,NaN,11.764718,6.721895,2.459116,0.000000,0.000000
2,NaN,NaN,15.973750,10.256377,4.487756,0.000000
3,NaN,NaN,NaN,20.764998,15.047878,8.189915
4,NaN,NaN,NaN,NaN,25.553642,20.764998
5,NaN,NaN,NaN,NaN,NaN,29.557914


In [ ]:
print('American Put value at date 0:', df.iloc[0,0].round(3))

American Put value at date 0: 8.367
